```{contents}
```
## **Node Handler in LangGraph**

A **Node Handler** in LangGraph is the **executable function bound to a node**, responsible for **reading the current graph state, performing computation (LLM calls, tool usage, logic, etc.), and returning partial state updates**.
It is the fundamental **unit of work** in LangGraph’s execution model.

---

### **1. Conceptual Role**

In LangGraph, **control flow is graph-based**, but **computation is function-based**.
A node exists only as a label until a **handler** is attached to it.

```
Graph Structure  →  Nodes  →  Node Handlers  →  State Transitions
```

> **Node Handler = State Transformer**

Formally:

[
f : State_{in} \rightarrow \Delta State
]

---

### **2. Handler Execution Contract**

Every handler follows a strict contract:

| Aspect       | Requirement                                   |
| ------------ | --------------------------------------------- |
| Input        | Receives current **State**                    |
| Output       | Returns **partial state update** (dict-like)  |
| Purity       | May be deterministic or stochastic            |
| Side effects | Allowed (LLM calls, APIs, tools)              |
| Persistence  | Updates merged into global state via reducers |

```python
def node_handler(state: State) -> dict:
    ...
    return {"field": new_value}
```

---

### **3. Why Partial State Updates Matter**

Handlers **do not replace** the state.
They emit only the fields they modify.

This enables:

* Concurrent updates
* Conflict resolution via reducers
* Transparent state evolution
* Checkpoint diffing & replay

---

### **4. Minimal Example**

```python
from typing import TypedDict
from langgraph.graph import StateGraph

class State(TypedDict):
    text: str
    tokens: int

def tokenize_handler(state):
    tokens = len(state["text"].split())
    return {"tokens": tokens}

builder = StateGraph(State)
builder.add_node("tokenizer", tokenize_handler)
```

Execution:

```
State before: {"text": "Hello world"}
Handler returns: {"tokens": 2}
State after: {"text": "Hello world", "tokens": 2}
```

---

### **5. Handler Categories**

| Type               | Purpose                 |
| ------------------ | ----------------------- |
| LLM Handler        | Calls language model    |
| Tool Handler       | Executes external tool  |
| Logic Handler      | Pure Python computation |
| Router Handler     | Selects next edge       |
| Human Handler      | Requests human input    |
| Validation Handler | Enforces constraints    |
| Error Handler      | Handles exceptions      |
| Retry Handler      | Re-executes on failure  |

---

### **6. LLM Handler Example**

```python
def reasoning_handler(state):
    response = llm.invoke(state["question"])
    return {"answer": response}
```

---

### **7. Router Handler**

Controls conditional edges:

```python
def router(state):
    if "error" in state:
        return "fix"
    return "final"
```

Used with:

```python
builder.add_conditional_edges("router", router, {
    "fix": "repair",
    "final": END
})
```

---

### **8. Handler with Side Effects & Tools**

```python
def web_search_handler(state):
    result = search_api(state["query"])
    log_event(result)
    return {"search_result": result}
```

---

### **9. Handler Safety & Production Controls**

| Mechanism     | Purpose           |
| ------------- | ----------------- |
| Timeout       | Prevent stalls    |
| Retries       | Fault tolerance   |
| Idempotency   | Safe re-execution |
| Validation    | State correctness |
| Logging       | Observability     |
| Checkpointing | Crash recovery    |

---

### **10. Handler Composition via Subgraphs**

Handlers may invoke **entire subgraphs**:

```python
def research_handler(state):
    sub_result = research_graph.invoke(state)
    return {"research": sub_result}
```

This enables **hierarchical systems**.

---

### **11. Mental Model**

Think of each node handler as a **microservice function** inside a distributed control system:

> **State enters → Handler transforms → State evolves → Graph routes**

This architecture allows LangGraph to model:

* Long-running agents
* Autonomous workflows
* Reliable production systems

### Demonstration

In [ ]:
from typing import TypedDict

class State(TypedDict):
    text: str
    tokens: int
    category: str
    result: str

def tokenize_handler(state: State):
    tokens = len(state["text"].split())
    return {"tokens": tokens}

def classify_handler(state: State):
    if state["tokens"] <= 5:
        return {"category": "short"}
    return {"category": "long"}

def router(state: State):
    return state["category"]

def short_handler(state: State):
    return {"result": "Text is short and simple."}

def long_handler(state: State):
    return {"result": "Text is long and complex."}



In [2]:
from langgraph.graph import StateGraph, END

builder = StateGraph(State)

builder.add_node("tokenize", tokenize_handler)
builder.add_node("classify", classify_handler)
builder.add_node("short", short_handler)
builder.add_node("long", long_handler)

builder.set_entry_point("tokenize")
builder.add_edge("tokenize", "classify")

builder.add_conditional_edges(
    "classify",
    router,
    {"short": "short", "long": "long"}
)

builder.add_edge("short", END)
builder.add_edge("long", END)

graph = builder.compile()


In [3]:
input_state = {"text": "LangGraph builds powerful agent systems"}
result = graph.invoke(input_state)
print(result)


{'text': 'LangGraph builds powerful agent systems', 'tokens': 5, 'category': 'short', 'result': 'Text is short and simple.'}
